### simulate data, get gripper position from camera parameters

In [ ]:

import os
import tensorflow as tf
import numpy as np
import pandas as pd
from tqdm import tqdm
import imageio.v2 as imageio  

# ==== Path ====
tfrecord_dir = "/data/gck/vla_planning/dataset/dexart/bucket_dex_art_dataset/1.0.0"
base_save_dir = "./dexart/bucket_png"
camera_views = [
    "bucket_viz"
]

os.makedirs(base_save_dir, exist_ok=True)

# ==== read tfrecord files ====
tfrecord_files = sorted([
    os.path.join(tfrecord_dir, f)
    for f in os.listdir(tfrecord_dir)
    if f.startswith("dex_art_dataset-train.tfrecord")
])

# ==== process each camera view ====
for view in camera_views:
    print(f"\nprocessing png: {view}")
    save_dir = os.path.join(base_save_dir, view)
    os.makedirs(save_dir, exist_ok=True)

    frame_counts = {}

    def parse_example(example_proto):
        feature_description = {
            f"steps/observation/{view}": tf.io.VarLenFeature(tf.string),
            "episode_metadata/file_path": tf.io.FixedLenFeature([], tf.string),
        }
        return tf.io.parse_single_example(example_proto, feature_description)

    for tfrecord_path in tqdm(tfrecord_files, desc=f"processing TFRecord - {view}"):
        raw_dataset = tf.data.TFRecordDataset(tfrecord_path)

        for raw_record in raw_dataset:
            example = parse_example(raw_record)
            episode_path = example["episode_metadata/file_path"].numpy().decode()
            episode_id = os.path.splitext(os.path.basename(episode_path))[0]

            try:
                frames = tf.sparse.to_dense(example[f"steps/observation/{view}"]).numpy()
            except:
                print(f"[skip] {episode_id} lack view {view}")
                continue

            if len(frames) == 0:
                continue

            frame_counts[episode_id] = len(frames)
            print(f"[episode] {episode_id} has {len(frames)} frame")

            # create folder for each episode 
            episode_dir = os.path.join(save_dir, episode_id)
            os.makedirs(episode_dir, exist_ok=True)

            for i, img_bytes in enumerate(frames):
                try:
                    img = tf.io.decode_png(img_bytes).numpy()
                    if img.shape[-1] == 4:
                        img = img[:, :, :3]  # remove alpha channel
                    frame_path = os.path.join(episode_dir, f"frame_{i:03d}.png")
                    imageio.imwrite(frame_path, img)
                except Exception as e:
                    print(f"[ekip eposide] {episode_id} # {i} frame error: {e}")

    # ==== atatus info ====
    frame_values = list(frame_counts.values())
    if frame_values:
        print(f"\n📊 frame count - view: {view}")
        print(f"total frame: {len(frame_values)}")
        print(f"average frame: {np.mean(frame_values):.2f}")
        print(f"maxiun frame: {np.max(frame_values)}")
        print(f"minium frame: {np.min(frame_values)}")

        df = pd.DataFrame({
            "episode_id": list(frame_counts.keys()),
            "frame_count": frame_values
        })
        csv_path = os.path.join(save_dir, f"{view}_frame_counts.csv")
        df.to_csv(csv_path, index=False)
        print(f"frame status save to: {csv_path}")
    else:
        print(f"No any valid frame was found - view: {view}")

print("\nall views successfully processed")


In [ ]:
# bucket total length is 33, gripper xyz position is 28:31
# other object 32 , xyz at -4:-1
import os
import json
import numpy as np
import tensorflow as tf
import transforms3d
from tqdm import tqdm
import re
import shutil
import glob
import imageio.v2 as imageio
import cv2

# === Path ===
tfrecord_dir = "/data/gck/vla_planning/dataset/dexart/bucket_dex_art_dataset/1.0.0"
video_dir = "./dexart/bucket_png/bucket_viz"
save_video_dir = "./dexart/results_gpos/bucket/vis_gpos"
save_json_path = "./dexart/results_gpos/bucket/gpos.json"

TARGET_RES = 224
CAMERA_RES = 1000
SCALE = TARGET_RES / CAMERA_RES

# === prepare for output dir ===
os.makedirs(save_video_dir, exist_ok=True)
for filename in os.listdir(save_video_dir):
    file_path = os.path.join(save_video_dir, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print(f'delete {file_path} false: {e}')

# === camera parameters ===
def get_bucket_extrinsics():
    position = np.array([0, 1, 0.5])
    quat = transforms3d.euler.euler2quat(np.pi / 2, np.pi, 0)
    R = transforms3d.quaternions.quat2mat(quat)
    t = position
    return R, t

def get_bucket_intrinsics(fov_deg=69.4):
    fov_rad = np.deg2rad(fov_deg)
    fx = fy = CAMERA_RES / (2 * np.tan(fov_rad / 2))
    cx = cy = CAMERA_RES / 2
    return np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])

def project_gripper_to_image(pos_world, K, R, t):
    x_cam = R @ (pos_world - t)
    x, y, z = x_cam
    if z <= 0:
        return [-1, -1]
    u = (K[0, 0] * x / z) + K[0, 2]
    v = (K[1, 1] * y / z) + K[1, 2]
    return [int(round(u)), int(round(v))]

# === TFRecord process ===
def parse_single_example(example_proto):
    feature_description = {
        "episode_metadata/file_path": tf.io.FixedLenFeature([], tf.string),
        "steps/state": tf.io.VarLenFeature(tf.float32),
    }
    return tf.io.parse_single_example(example_proto, feature_description)

# === main pipeline ===
def run():
    K = get_bucket_intrinsics()
    R, t = get_bucket_extrinsics()
    uv_scaled_dict = {}

    tfrecord_files = sorted([
        os.path.join(tfrecord_dir, f)
        for f in os.listdir(tfrecord_dir)
        if f.endswith(".tfrecord") or ".tfrecord-" in f
    ])

    for tf_file in tqdm(tfrecord_files, desc="processing TFRecords"):
        dataset = tf.data.TFRecordDataset(tf_file)
        for raw_record in dataset:
            example = parse_single_example(raw_record)
            file_path = example["episode_metadata/file_path"].numpy().decode()
            state_seq = tf.sparse.to_dense(example["steps/state"]).numpy()

            if len(state_seq) == 0:
                continue

            try:
                states = state_seq.reshape(-1, 33)
            except:
                continue

            gripper_pos_seq = states[:, 28:31]
            uv_full = [project_gripper_to_image(p, K, R, t) for p in gripper_pos_seq]
            uv_scaled = [[int(u * SCALE), int(v * SCALE)] if u >= 0 and v >= 0 else [-1, -1] for u, v in uv_full]

            match = re.search(r"episode_(\d+)_combined\.npz$", file_path)
            if not match:
                continue
            episode_id = match.group(1)
            uv_scaled_dict[episode_id] = uv_scaled

            # === read from folder containing PNG===
            frame_dir = os.path.join(video_dir, f"episode_{episode_id}_combined")

            if not os.path.exists(frame_dir):
                print(f"[skip] No image folder: {frame_dir}")
                continue

            frame_paths = sorted(glob.glob(os.path.join(frame_dir, "frame_*.png")))
            if len(frame_paths) == 0:
                print(f"[skip] NO image: {frame_dir}")
                continue

            fps = 10
            frame_count = len(frame_paths)
            print(f"[check] {episode_id}: states frame={states.shape[0]}, image frame={frame_count}")

            # output video folder （double check process quality)
            out_path = os.path.join(save_video_dir, f"{episode_id}_vis.mp4")
            frame0 = imageio.imread(frame_paths[0])
            h, w = TARGET_RES, TARGET_RES
            out = cv2.VideoWriter(out_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

            for frame_idx, frame_path in enumerate(frame_paths):
                if frame_idx >= len(uv_scaled):
                    break

                frame = imageio.imread(frame_path)
                frame = cv2.resize(frame, (TARGET_RES, TARGET_RES))

                # === red point： gripper position  ===
                u, v = uv_scaled[frame_idx]
                u = max(0, min(u, TARGET_RES - 1))
                v = max(0, min(v, TARGET_RES - 1))
                cv2.circle(frame, (u, v), radius=4, color=(0, 0, 255), thickness=-1)

                pos_xyz = gripper_pos_seq[frame_idx]
                text = f"x={pos_xyz[0]:.3f}, y={pos_xyz[1]:.3f}, z={pos_xyz[2]:.3f}"
                cv2.putText(frame, text, (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 0, 255), 1)

                # === green point：origin point ===
                origin_uv = project_gripper_to_image(np.array([0, 0, 0]), K, R, t)
                if origin_uv[0] >= 0 and origin_uv[1] >= 0:
                    origin_uv_scaled = [int(origin_uv[0] * SCALE), int(origin_uv[1] * SCALE)]
                    cv2.circle(frame, tuple(origin_uv_scaled), radius=4, color=(0, 255, 0), thickness=-1)
                    cv2.putText(frame, "origin", (origin_uv_scaled[0] + 5, origin_uv_scaled[1] - 5),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0), 1)

                out.write(frame)

            out.release()
            print(f"[video save] {out_path}")

    with open(save_json_path, "w") as f:
        json.dump(uv_scaled_dict, f, indent=2)
    print(f"[JSON output] {save_json_path}")

# === execuate ===
run()


### real data

In [ ]:
import os
import cv2
import numpy as np
import json
from tqdm import tqdm
import matplotlib.pyplot as plt
from scipy.interpolate import UnivariateSpline
from scipy.signal import savgol_filter



VIDEO_FPS = 15
DRAW_RADIUS = 4
BOTTOM_HEIGHT = 40
SPLINE_SMOOTH = 50






def get_color_frame_path(mask_fname):
    digits = ''.join(filter(str.isdigit, mask_fname))
    return os.path.join(COLOR_FRAME_DIR, f"annotated_{digits.zfill(5)}.jpg")




# ========== cut mask ==========
def truncate_by_bottom_curve(mask, bottom_height=40, poly_order=1):
    h, w = mask.shape
    bottom_region = mask[h - bottom_height :, :]
    ys, xs = np.where(bottom_region > 0)
    if len(xs) < 20:
        return mask.copy(), None

    xs_global = xs
    ys_global = ys + (h - bottom_height)

    coeffs = np.polyfit(xs_global, ys_global, deg=poly_order)
    poly = np.poly1d(coeffs)
    fitted_y = poly(np.arange(w)).astype(np.int32)

    truncated_mask = np.zeros_like(mask)
    for x in range(w):
        y_limit = fitted_y[x]
        if 0 <= y_limit < h:
            truncated_mask[:y_limit, x] = mask[:y_limit, x]

    return truncated_mask, fitted_y

# ========== middle point ==========
def get_bottom_center(mask, margin=3):
    ys, xs = np.where(mask > 0)
    if len(ys) == 0:
        return None
    bottom_y = np.max(ys)
    xs_bottom = xs[ys >= bottom_y - margin]
    if len(xs_bottom) == 0:
        return None
    center_x = int(np.mean(xs_bottom))
    return (center_x, bottom_y)

# ========== spline fitting ==========
def smooth_x_with_spline(center_list, spline_smooth=20):
    valid_ts, valid_xs = [], []
    for t, pt in enumerate(center_list):
        if pt is not None:
            valid_ts.append(t)
            valid_xs.append(pt[0])

    if len(valid_ts) < 5:
        print("not enough valid frames")
        return center_list, None, None

    spline = UnivariateSpline(valid_ts, valid_xs, s=spline_smooth)
    smoothed, smooth_xs = [], []

    for t, pt in enumerate(center_list):
        if pt is not None:
            smooth_x = int(spline(t))
            smoothed.append((smooth_x, pt[1]))
            smooth_xs.append(smooth_x)
        else:
            smoothed.append(None)
            smooth_xs.append(None)

    return smoothed, valid_ts, smooth_xs


def get_max_continuous_width(row):
    """
    input a row of pixels, return the width of the longest continuous non-zero segment
    """
    max_len = 0
    current_len = 0
    for val in row:
        if val > 0:
            current_len += 1
            max_len = max(max_len, current_len)
        else:
            current_len = 0
    return max_len



def get_max_continuous_width(row):
    max_len = 0
    curr_len = 0
    for val in row:
        if val > 0:
            curr_len += 1
            max_len = max(max_len, curr_len)
        else:
            curr_len = 0
    return max_len



from scipy.interpolate import UnivariateSpline
import numpy as np

def get_max_continuous_width(row):
    max_len = 0
    curr_len = 0
    for val in row:
        if val > 0:
            curr_len += 1
            max_len = max(max_len, curr_len)
        else:
            curr_len = 0
    return max_len

def remove_fingers_by_jump(mask, 
                           strategy='avg', 
                           drop_ratio_thresh=0.4, 
                           min_valid_width=10, 
                           return_jump=False):
    """
    jump detection and removal of finger regions, supports three strategies:
    - 'avg': compare the average width of the first 3 vs the last 3 rows
    - 'diff': maximum descending derivative
    - 'spline': spline curve minimum point
    """
    h, w = mask.shape
    cleaned = mask.copy()
    row_widths = [get_max_continuous_width(mask[y, :]) for y in range(h)]
    jump_row = None

    if strategy == 'avg':
        for y in range(3, h - 3):
            prev_avg = np.mean(row_widths[y - 3:y])
            curr_avg = np.mean(row_widths[y:y + 3])
            if prev_avg >= min_valid_width and curr_avg < prev_avg * (1 - drop_ratio_thresh):
                jump_row = y
                break

    elif strategy == 'diff':
        diffs = np.diff(row_widths)
        jump_row = np.argmin(diffs)
        if row_widths[jump_row] < min_valid_width:
            jump_row = None

    elif strategy == 'spline':
        x = np.arange(h)
        y = np.array(row_widths)
        spline = UnivariateSpline(x, y, s=10)
        second_deriv = spline.derivative(n=2)(x)
        jump_row = np.argmin(second_deriv)
        if row_widths[jump_row] < min_valid_width:
            jump_row = None

    if jump_row is not None:
        cleaned[jump_row:, :] = 0

    if return_jump:
        return cleaned, row_widths, jump_row
    else:
        return cleaned





import matplotlib.pyplot as plt
import numpy as np

def plot_row_width_profile(mask, title="Row-wise Width Profile"):
    """
    calculate and plot the horizontal width curve of each row (width of white connected pixels).
    """
    h, w = mask.shape
    row_widths = []

    for y in range(h):
        width = get_max_continuous_width(mask[y, :])

        row_widths.append(width)


import cv2
import numpy as np
import matplotlib.pyplot as plt

def overlay_row_width_curve(mask, scale=2.0, curve_color=(0, 255, 255)):
    """
    overlay the horizontal width curve of each row (vertical alignment) on the right side of the mask image.
    return the new image with overlay (BGR).
    """
    h, w = mask.shape
    mask_vis = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)

    # === calculate the width of each row ===
    row_widths = []
    max_width = 1  # prevent division by zero
    for y in range(h):
        xs = np.where(mask[y, :] > 0)[0]
        width = get_max_continuous_width(mask[y, :])
        row_widths.append(width)
        max_width = max(max_width, width)

    # === construct the side width image ===
    curve_width = int(max_width * scale)
    side_curve = np.ones((h, curve_width, 3), dtype=np.uint8) * 0  # black background

    for y, width in enumerate(row_widths):
        x_pos = int(width * scale)
        cv2.line(side_curve, (0, y), (x_pos, y), curve_color, 1)

    # === concatenate the images ===
    combined = np.concatenate((mask_vis, side_curve), axis=1)
    return combined



# ========== show demo ==========
def show_demo_image():
    
    img = cv2.imread(DEMO_IMAGE_PATH, cv2.IMREAD_GRAYSCALE)
    _, binary = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
    truncated, fitted_y = truncate_by_bottom_curve(binary, BOTTOM_HEIGHT)
    truncated = remove_fingers_by_jump(truncated)



    center = get_bottom_center(truncated)

    vis = cv2.cvtColor(truncated, cv2.COLOR_GRAY2BGR)
    if center:
        cv2.circle(vis, center, DRAW_RADIUS, (0, 0, 255), -1)
    if fitted_y is not None:
        for x, y in enumerate(fitted_y):
            if 0 <= y < vis.shape[0]:
                vis[y, x] = (0, 255, 0)



    


# ========== generate video ==========
def generate_video():
    def extract_frame_number(filename):
        digits = ''.join(filter(str.isdigit, filename))
        return int(digits) if digits else float('inf')

    image_files = sorted([
        f for f in os.listdir(INPUT_DIR)
        if f.startswith("mask") and f.endswith(".jpg") and any(c.isdigit() for c in f)
    ], key=extract_frame_number)

    if not image_files:
        raise ValueError("No mask images found.")

    first_img = cv2.imread(os.path.join(INPUT_DIR, image_files[0]), cv2.IMREAD_GRAYSCALE)
    h, w = first_img.shape
    video_writer = cv2.VideoWriter(OUTPUT_VIDEO, cv2.VideoWriter_fourcc(*'mp4v'), VIDEO_FPS, (w, h))

    raw_centers, fitted_lines = [], []

    for fname in tqdm(image_files, desc="Preprocessing"):
        path = os.path.join(INPUT_DIR, fname)
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        _, binary = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
        truncated, fitted_y = truncate_by_bottom_curve(binary, BOTTOM_HEIGHT)
        truncated = remove_fingers_by_jump(truncated) 
        
        save_path = os.path.join(FILTERED_MASK_DIR, fname)
        cv2.imwrite(save_path, truncated)


        center = get_bottom_center(truncated)
        raw_centers.append(center)
        fitted_lines.append(fitted_y)

    smoothed_centers, valid_ts, smooth_xs = smooth_x_with_spline(raw_centers, SPLINE_SMOOTH)
    # === Savitzky-Golay smoothing ===
    # fill in missing values (use the nearest non-empty value to fill) to avoid None breaking the filter
    smooth_xs_array = np.array([x if x is not None else 0 for x in smooth_xs])
    # set the window and order (must be an odd window, poly_order < window)
    window_length = min(11, len(smooth_xs_array) - (1 - len(smooth_xs_array) %   2))  # 保证奇数窗口
    smooth_xs_filtered = savgol_filter(smooth_xs_array, window_length=window_length, polyorder=2)

    TAIL_N = 10
    valid_ts = [i for i, pt in enumerate(smoothed_centers) if pt is not None]

    for i in valid_ts:
        pt = smoothed_centers[i]
        idx = valid_ts.index(i)

        if idx < TAIL_N or idx >= len(valid_ts) - TAIL_N:
            smoothed_centers[i] = pt  # 原始值
        else:
            smoothed_centers[i] = (int(smooth_xs_filtered[i]), pt[1])

    # ==== x curve ====
    raw_xs = [pt[0] if pt else None for pt in raw_centers]


    center_dict = {}
    for i, fname in enumerate(tqdm(image_files, desc="Rendering")):
        color_path = get_color_frame_path(fname)
        if os.path.exists(color_path):
            vis = cv2.imread(color_path)
        else:
            print(f"No color image: {color_path}")
            vis = np.zeros((h, w, 3), dtype=np.uint8)

        center = smoothed_centers[i]
        fitted_y = fitted_lines[i]

        # === draw the frame number in the top left corner ===
        frame_number = ''.join(filter(str.isdigit, fname))
        text = f"Frame: {frame_number}"
        cv2.putText(vis, text, (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 2)


        if center:
            cv2.circle(vis, center, DRAW_RADIUS, (0, 0, 255), -1)
            center_dict[fname] = [int(center[0]), int(center[1])]
        else:
            center_dict[fname] = None

        if fitted_y is not None:
            for x, y in enumerate(fitted_y):
                if 0 <= y < h:
                    vis[y, x] = (0, 255, 0)

        video_writer.write(vis)

    video_writer.release()
    with open(OUTPUT_JSON, 'w') as f:
        json.dump(center_dict, f, indent=2)

    print(f"video saved to: {OUTPUT_VIDEO}")
    print(f"center coordinates saved to: {OUTPUT_JSON}")

# ========== execute ==========
if __name__ == "__main__":
    # show_demo_image()
    # generate_video()
    from pathlib import Path

    ROOT_DIR = Path("path/to//grounded_sam2/output/mask/dir")
    all_gripper_dirs = sorted(ROOT_DIR.glob("*/white gripper"))

    for gripper_dir in tqdm(all_gripper_dirs, desc="Batch Processing"):
        INPUT_DIR = str(gripper_dir)
        COLOR_FRAME_DIR = str(gripper_dir.parent)
        OUTPUT_VIDEO = os.path.join(INPUT_DIR, "gripper_center_truncated.mp4")
        OUTPUT_JSON = os.path.join(INPUT_DIR, "gripper_centers_truncated.json")
        FILTERED_MASK_DIR = os.path.join(os.path.dirname(INPUT_DIR), "filtered_mask")
        os.makedirs(FILTERED_MASK_DIR, exist_ok=True)

        # 自动选择 demo image
        mask_files = sorted([f for f in os.listdir(INPUT_DIR) if f.startswith("mask_") and f.endswith(".jpg")])
        if not mask_files:
            print(f"{INPUT_DIR} no mask image, skip")
            continue
        DEMO_IMAGE_PATH = os.path.join(INPUT_DIR, mask_files[len(mask_files) // 2])

        print(f"\nprocessing: {INPUT_DIR}")
        show_demo_image()
        generate_video()



In [ ]:
# post process gpos
import os
import json
from tqdm import tqdm

VIS_VIDEOS_DIR = "path/to/vis_videos_dir"
OUTPUT_PATH = "path/to/gripper_positions.json"

gripper_positions_all = {}

for episode_folder in tqdm(sorted(os.listdir(VIS_VIDEOS_DIR)), desc="Processing episodes"):
    episode_path = os.path.join(VIS_VIDEOS_DIR, episode_folder)
    gripper_json_path = os.path.join(episode_path, "white gripper", "gripper_centers_truncated.json")  #修改white gripper字段

    if not os.path.exists(gripper_json_path):
        print(f"Missing {gripper_json_path}")
        continue

    with open(gripper_json_path, "r") as f:
        gripper_data = json.load(f)

    frame_keys = sorted(gripper_data.keys(), key=lambda x: int(''.join(filter(str.isdigit, x))))
    gripper_positions = [gripper_data[k] for k in frame_keys]

    episode_id = f"{episode_folder.split('_')[0]}"
    gripper_positions_all[episode_id] = gripper_positions

os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)
with open(OUTPUT_PATH, "w") as f:
    json.dump(gripper_positions_all, f)

print(f"Saved gripper_positions.json to {OUTPUT_PATH}")
